# Uploading Modules

In [1]:
import pandas as pd
import nltk
import re
import numpy as np
from wordcloud import WordCloud
import gensim
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

# Importing Data

In [2]:
active20=pd.read_csv("active20-clean.csv")
active19=pd.read_csv("active19-clean.csv")
lazy20=pd.read_csv("lazy20-clean.csv")
lazy19=pd.read_csv("lazy19-clean.csv")
twitter_users = pd.read_csv("Twitter users.csv")

in this section we will try to answer the question: <br>
What Did Active and Lazy User Cohorts Discuss on Twitter in Spring 2019 and in Spring 2020?<br>
To answer this question, we will use topic modeling. Topic modeling is an unsupervised machine learning technique. This means it can infer patterns and cluster similar expressions without needing to define topic tags or train data beforehand. We will use this approach to identify main topics of dicsussion in 2019 and 2020 in both cohorts. Based on the EDA findings, we expect to see weather-related topic in active 2019 tweets, game-related topics in lazy 2019 tweets and covid-related topic in both active and lazy in 2020. 

Topic modeling algorithms are statistical methods that analyze the words of documents to discover the themes that pervade a large collection of documents. The basic idea of topic modeling is that a document is a mixture of latent topics and each topic is expressed by a distribution of words.

# Latent Dirichlet Allocation (LDA) Model in gensim

Latent Dirichlet Allocation (LDA) is the most popular topic modeling method in the field of text mining. The output of LDA provides two probability matrices: 1) the (posterior) probability distribution of each document over the topics, and 2) the probability distribution of words in a given topic. The calculated probability matrixes are used to make inference about the topics and documents for text mining. LDA has been shown to be an effective tool for text mining of large datasets. We will then do topic model-derived clustering based on highest probable topic assignment.

In [4]:
# creating list of documents from the preprocessed tweets:
active19_docs=active19.clean_text.dropna()
active19_docs=active19_docs.to_list()
active20_docs=active20.clean_text.dropna()
active20_docs=active20_docs.to_list()
lazy19_docs=lazy19.clean_text.dropna()
lazy19_docs=lazy19_docs.to_list()
lazy20_docs=lazy20.clean_text.dropna()
lazy20_docs=lazy20_docs.to_list()

In [5]:
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

# tokenize the tweet docs (we lemmatize the words in tweet docs 
# in order to reduce inflectional forms to a common base form):
active19_docs_tokenized = [word_tokenize(doc) for doc in active19_docs]
active20_docs_tokenized = [word_tokenize(doc) for doc in active20_docs]
lazy19_docs_tokenized = [word_tokenize(doc) for doc in lazy19_docs]
lazy20_docs_tokenized = [word_tokenize(doc) for doc in lazy20_docs]

# make a gensim dictionary with tokenized tweet docs:
dict_active19 = Dictionary(active19_docs_tokenized)
dict_active20 = Dictionary(active20_docs_tokenized)
dict_lazy19 = Dictionary(lazy19_docs_tokenized)
dict_lazy20 = Dictionary(lazy20_docs_tokenized)

# create a gensim corpus (different from the regular corpus in so that 
# each document is converted to a bag of words using token ids):
active19_corpus = [dict_active19.doc2bow(doc) for doc in active19_docs_tokenized] 
active20_corpus = [dict_active20.doc2bow(doc) for doc in active20_docs_tokenized] 
lazy19_corpus = [dict_lazy19.doc2bow(doc) for doc in lazy19_docs_tokenized] 
lazy20_corpus = [dict_lazy20.doc2bow(doc) for doc in lazy20_docs_tokenized] 

# how to go to the word behind a token
# dict_active19.token2id

In [5]:
active19_docs_tokenized[0]

['new', 'knife', 'post', 'forum', 'new', 'opinel', 'httpstcoprbmwht', 'knives']

In [6]:
# TD-IDF model to define word frequency:
from gensim import corpora, models

tfidf_active19 = models.TfidfModel(active19_corpus)
corpus_tfidf_active19 = tfidf_active19[active19_corpus]
#from pprint import pprint
#for doc in corpus_tfidf_active19:
#    pprint(doc)
#    break
tfidf_active20 = models.TfidfModel(active20_corpus)
corpus_tfidf_active20 = tfidf_active20[active20_corpus]

tfidf_lazy19 = models.TfidfModel(lazy19_corpus)
corpus_tfidf_lazy19 = tfidf_lazy19[lazy19_corpus]

tfidf_lazy20 = models.TfidfModel(lazy20_corpus)
corpus_tfidf_lazy20 = tfidf_lazy20[lazy20_corpus]

### Major Topics for Active Users

In [7]:
# Running LDA using TF-IDF

# Active 19
lda_model_tfidf_active19 = gensim.models.LdaMulticore(corpus_tfidf_active19, num_topics=3, id2word=dict_active19, passes =3)

In [8]:
for idx, topic in lda_model_tfidf_active19.print_topics(num_topics=-1,num_words=25):
    print('Topic: {} \n Word: {}\n'.format(idx, topic))

Topic: 0 
 Word: 0.001*"new" + 0.001*"facebook" + 0.001*"check" + 0.001*"see" + 0.001*"video" + 0.001*"great" + 0.001*"not" + 0.001*"posted" + 0.001*"get" + 0.001*"love" + 0.001*"one" + 0.001*"us" + 0.001*"day" + 0.001*"week" + 0.001*"today" + 0.001*"come" + 0.001*"spring" + 0.001*"ready" + 0.001*"weekend" + 0.001*"join" + 0.001*"morning" + 0.001*"go" + 0.001*"like" + 0.001*"work" + 0.001*"find"

Topic: 1 
 Word: 0.001*"spring" + 0.001*"great" + 0.001*"new" + 0.001*"one" + 0.001*"us" + 0.001*"get" + 0.001*"love" + 0.001*"time" + 0.001*"check" + 0.001*"happy" + 0.001*"day" + 0.001*"please" + 0.001*"fun" + 0.001*"see" + 0.001*"video" + 0.001*"last" + 0.001*"going" + 0.001*"like" + 0.001*"life" + 0.001*"live" + 0.001*"back" + 0.001*"good" + 0.001*"night" + 0.001*"today" + 0.001*"dont"

Topic: 2 
 Word: 0.001*"day" + 0.001*"new" + 0.001*"great" + 0.001*"time" + 0.001*"hiking" + 0.001*"check" + 0.001*"today" + 0.001*"april" + 0.001*"good" + 0.001*"video" + 0.001*"one" + 0.001*"best" + 0.001

As seen from LDA model with term frequency, in 2019 the active users Twitter topics can be split into 3 groups:<br>
1) Happy time hiking <br>
2) Social media (Facebook, videos) about the weekend fun outdoors  <br> 
3) Social media (Facebook, videos) about camping and the Spring season <br>
However, term frequency was almost the same, and none of the words in any topic were particularly important.

In [9]:
# Active 20
lda_model_tfidf_active20 = gensim.models.LdaMulticore(corpus_tfidf_active20, num_topics=3, id2word=dict_active20, passes=3)

In [15]:
for idx, topic in lda_model_tfidf_active20.print_topics(num_topics=-1,num_words=25):
    print('Topic: {} \n Word: {}\n'.format(idx, topic))

Topic: 0 
 Word: 0.002*"covid" + 0.001*"day" + 0.001*"us" + 0.001*"time" + 0.001*"coronavirus" + 0.001*"good" + 0.001*"not" + 0.001*"today" + 0.001*"people" + 0.001*"get" + 0.001*"great" + 0.001*"please" + 0.001*"one" + 0.001*"home" + 0.001*"stay" + 0.001*"new" + 0.001*"help" + 0.001*"like" + 0.001*"happy" + 0.001*"morning" + 0.001*"need" + 0.001*"going" + 0.001*"work" + 0.001*"best" + 0.001*"still"

Topic: 1 
 Word: 0.001*"new" + 0.001*"one" + 0.001*"us" + 0.001*"not" + 0.001*"day" + 0.001*"time" + 0.001*"today" + 0.001*"home" + 0.001*"check" + 0.001*"camp" + 0.001*"like" + 0.001*"covid" + 0.001*"love" + 0.001*"get" + 0.001*"good" + 0.001*"coronavirus" + 0.001*"social" + 0.001*"live" + 0.001*"family" + 0.001*"video" + 0.001*"go" + 0.001*"via" + 0.001*"work" + 0.001*"see" + 0.001*"keep"

Topic: 2 
 Word: 0.001*"week" + 0.001*"see" + 0.001*"latest" + 0.001*"twitter" + 0.001*"via" + 0.001*"thanks" + 0.001*"photo" + 0.001*"stayindoorsdreamoutdoors" + 0.001*"daily" + 0.001*"party_popper" +

The 3 Twitter topics in 2020 among active users:<br>
1) Coronavirus staying at home and trying to stay positive (us, best, great)<br>
2) Coronavirus keeping in touch via social media (family, video, social) <br>
3) Missing outdoors during Coronavirus (#stayindoorsdreamoutdoors)<br>
All the words were about the same term frequency, so the topic themes are not very evident.

### Major Topics for Lazy Users

In [11]:
# Lazy 19
lda_model_tfidf_lazy19 = gensim.models.LdaMulticore(corpus_tfidf_lazy19, num_topics=3, id2word=dict_lazy19, passes =3)

In [16]:
for idx, topic in lda_model_tfidf_lazy19.print_topics(num_topics=-1,num_words=25):
    print('Topic: {} \n Word: {}\n'.format(idx, topic))

Topic: 0 
 Word: 0.001*"like" + 0.001*"one" + 0.001*"new" + 0.001*"love" + 0.001*"day" + 0.001*"get" + 0.001*"not" + 0.001*"today" + 0.001*"never" + 0.001*"lol" + 0.001*"even" + 0.001*"think" + 0.001*"got" + 0.001*"check" + 0.001*"live" + 0.001*"work" + 0.001*"little" + 0.001*"going" + 0.001*"anyone" + 0.001*"th" + 0.001*"us" + 0.001*"go" + 0.001*"know" + 0.001*"twitter" + 0.001*"time"

Topic: 1 
 Word: 0.002*"new" + 0.001*"not" + 0.001*"going" + 0.001*"happy" + 0.001*"life" + 0.001*"week" + 0.001*"day" + 0.001*"get" + 0.001*"time" + 0.001*"like" + 0.001*"one" + 0.001*"free" + 0.001*"last" + 0.001*"please" + 0.001*"love" + 0.001*"bestbuyatplaylistlive" + 0.001*"bestbuy" + 0.001*"teamcanon" + 0.001*"ever" + 0.001*"people" + 0.001*"much" + 0.001*"hey" + 0.001*"best" + 0.001*"year" + 0.001*"know"

Topic: 2 
 Word: 0.002*"college" + 0.002*"hate" + 0.001*"not" + 0.001*"love" + 0.001*"back" + 0.001*"one" + 0.001*"time" + 0.001*"newprofilepic" + 0.001*"see" + 0.001*"dont" + 0.001*"need" + 0.0

In 2019 the top 3 topics for lazy users were:<br>
1) General Twitter chatter  <br>
2) Best Buy Playlist Live event <br>
3) Not enjoying college <br>

In [13]:
# Lazy 20
lda_model_tfidf_lazy20 = gensim.models.LdaMulticore(corpus_tfidf_lazy20, num_topics=3, id2word=dict_lazy20, passes =3)

In [17]:
for idx, topic in lda_model_tfidf_lazy20.print_topics(num_topics=-1,num_words=30):
    print('Topic: {} \n Word: {}\n'.format(idx, topic))

Topic: 0 
 Word: 0.001*"like" + 0.001*"good" + 0.001*"need" + 0.001*"week" + 0.001*"twitter" + 0.001*"today" + 0.001*"see" + 0.001*"get" + 0.001*"not" + 0.001*"day" + 0.001*"covid" + 0.001*"stay" + 0.001*"right" + 0.001*"time" + 0.001*"work" + 0.001*"home" + 0.001*"love" + 0.001*"say" + 0.001*"feel" + 0.001*"people" + 0.001*"one" + 0.001*"social" + 0.001*"red_heart" + 0.001*"going" + 0.001*"party_popper" + 0.001*"thanks" + 0.001*"life" + 0.001*"go" + 0.001*"happy" + 0.001*"god"

Topic: 1 
 Word: 0.002*"not" + 0.002*"want" + 0.002*"miss" + 0.001*"going" + 0.001*"like" + 0.001*"go" + 0.001*"really" + 0.001*"na" + 0.001*"quarantine" + 0.001*"day" + 0.001*"one" + 0.001*"wan" + 0.001*"get" + 0.001*"hair" + 0.001*"love" + 0.001*"people" + 0.001*"time" + 0.001*"got" + 0.001*"know" + 0.001*"face_with_tears_of_joy" + 0.001*"fuck" + 0.001*"home" + 0.001*"new" + 0.001*"today" + 0.001*"newprofilepic" + 0.001*"back" + 0.001*"good" + 0.001*"work" + 0.001*"look" + 0.001*"check"

Topic: 2 
 Word: 0.00

In 2020 the top 3 topics for lazy users were:<br>
1) Covid support and activities (home, red_heart, party_pooper)<br>
2) Quarantine emotions (face_with_tears_of_joy, miss) <br>
3) Quarantine activities (playing Animal Crossing and Nintendo Switch) <br>

# Non-Negative Matrix Factorization (NMF) in sklearn

Unlike LDA, NMF relies on linear algebra. It approximates a nonnegative matrix by the product of two low-rank nonnegative matrices. Since it gives semantically meaningful result that is easily interpretable in clustering applications, NMF has been widely used as a clustering method especially for document data, and as a topic modeling method.

In [59]:
# Run NMF
active19_nmf = NMF(n_components=3).fit(active19_tfidf) #, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd'
active20_nmf = NMF(n_components=3).fit(active20_tfidf)
lazy19_nmf = NMF(n_components=3).fit(lazy19_tfidf)
lazy20_nmf = NMF(n_components=3).fit(lazy20_tfidf)

In [37]:
# create a function to display topics
def display_topics(docs, model, no_top_words):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf = tfidf_vectorizer.fit_transform(docs)
    feature_names = tfidf_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Major Topics for Active Users

In [51]:
# active 2019:
display_topics(active19_docs, active19_nmf, 30)

Topic 0:
facebook posted video new httpstcovxvtekam httpstcoltxkwnxca httpstcoovmahqwpnb httpstcobtfmshya httpstcoktntegrafp httpstcovzwgxtrz httpstcomjcwhnr httpstcoboppqyuy httpstcokxmzfjlo httpstcopjrjhty httpstcocxufmhhli httpstcolqguotxcq httpstcobstkcql httpstcobcwbiqel httpstcosqaptxw httpstcoslwobsjq httpstcoqxzcihjfy httpstcojtdconsund httpstcosquanfmu httpstcoxcaiqoipl httpstcoqgkjcrlpra httpstcoqypphvvb httpstcokqcbdxiiyi httpstcokbjbztjyb httpstcoalmyutmq httpstcoaahgpow
Topic 1:
great day us one time check spring get today see love like go april good weekend season not new th hiking come best happy join fun first friends looking week
Topic 2:
camp sagar rajasthan chhatra india httpstcooxavwukax luxurycamps luxurycamp wilderness around walk nature tent spring httpstcooxavwuzyv campseriessentelevisioncom theshowdownsen theshowdown regional invited neelgai film dungari tents moti email chance httpstcoluuyjti httpstcohkxwyzb httpstcohpmruymf


The top 3 topics for active users in 2019 were:<br>
1) Links to Facebook videos<br>
2) Great time hiking in the Spring<br>
3) Advertising of camping in India by chhatrasagar Twitter user

In [50]:
# active 2020:
display_topics(active20_docs, active20_nmf, 30)

Topic 0:
timeout usba dayathewoods getbetter homebivouac nothingbasically todaysnake onelebanonngo stayfit covidaustralia goodcall lildickytweets newcollection greatdaysout pepperelleddie helpeveryone safleoedd knowturning negative workgloves plesently goalie makesiteasier goingwell keephawaiicooking back_arrow lovefife thankyoupolis wellness coronavirusbill
Topic 1:
two_heartstwo_heartstwo_hearts readindie partying_faceparty_popper weeklymealplan seeeeee revenant lily mera revered mepolitics representation representationmatters fanswhich biggreenegg thankyoupolis vickibrightwood httpstcogrpoq followsclapping_handsclapping_handsclapping_hands httpstcoknrjdolfle httpstcotheotgzs httpstcoddtgwup httpstcozhsvxmwqq httpstcokjpcz httpstcolwcyzpsafo httpstcotftlkkkxx httpstcotztcwzz httpstcofvhqsp httpstcokcyroklfa httpstcoeqbzvpme httpstcotvtkihysgc
Topic 2:
latulipmike dailydoseofgreenspace thatchedroof newsreading coronavirusbill covidaustralia outdoorstyle hikinga spouses huntingusa wild

The top 3 topics for active users in 2020 were:<br>
1) Coronavirus (stay at home, wellness, #getbetter, etc.) <br>
2) Thanking people (emojis of clapping_hands,two_hearts, #thankopolis, etc.)<br>
3) Activities during Coronavirus (newsreading, hiking, camping, #dailydoseofgreenspace)

### Major Topics for Active Users

In [55]:
# lazy 2019:
display_topics(lazy19_docs, lazy19_nmf, 30)

Topic 0:
college hate fucking really httpstcoeagccrgpvd beach reason song another curiouscat deactivate likely httpstcoevffrco ur youll ima game send due anymore reminded intro woman_facepalming_light_skin_tone truth thrones brain done enrolled face_with_rolling_eyesface_with_rolling_eyesface_with_rolling_eyes medication
Topic 1:
not one love new day like get going back time people want see today never life dont know us go need got much best year first happy feel great good
Topic 2:
bestbuyatplaylistlive bestbuy teamcanon win let daughter meetup httpstcorgylmtmvr httpstcocrctdgit giveaway enter entered pusheen tiger roll gift card httpstconiudjkvdxs fearful_face cash national grand giant pc hope httpstcoatphnri httpstcoptdjmnah boosttiger tsampcs shop


The top 3 topics for lazy users in 2019 were:<br>
1) Hating college (hate, college, face_with_rolling_eyes emoji, etc.) <br>
2) Good day feeling great (best, day, happy, great, etc.)<br>
3) Twitter contests (bestbuyatplaylistlive, gift card, meetup, etc.)

In [65]:
# lazy 2020:
display_topics(lazy20_docs, lazy20_nmf, 30)

Topic 0:
nouns lincoln tinuod gettingbetter golfing periodpiece onte kokoresign reclaimingmyhealth godblessthedishwasher goodreads homeorwork watchedrewatched lowemissions nemesisabitch backe sektorunda wottitotsclub stops riskchronicles thought_balloon wt govermentface_with_symbols_on_mouthface_with_symbols_on_mouthface_with_symbols_on_mouth multiplexes felineboyfriend everybody malaysianpolitics doomed newyorklockdown stefanlöfven
Topic 1:
acnhdesign animalcrossingdodocode noch lowemissions issued cutoff animalfacts longsmiling_face_with_heart govermentface_with_symbols_on_mouthface_with_symbols_on_mouthface_with_symbols_on_mouth shockandawe httpstconisfz httpstcobymckdgsy httpstcoevprpgnn findings how wolves nextyear animalcrossingdesign harapkan volqx oliviareidxo overflow httpstcodxjgyxgwd enright multiplexes httpstcoucyvkxaa thedarkknightrises crowned yorme gamitin
Topic 2:
daylong queensono tomatoes evil_monkey harapkan goodreads maggid homeorwork motivating wottitotsclub onte b

The top 3 topics for lazy users in 2020 were:<br>
1) Lockdown and staying at home (#homeorwork, #reclaimingmyhealth, #newyorklockdown, etc.) <br>
2) Playing online games (#achndesign, animalcrossing, #gameitin, etc.)<br>
3) Other activities (goodreads, #homeorwork, #daylong, etc.)